In [ ]:
import numpy as np
import firedrake
from firedrake import Constant, inner, grad, dx
import matplotlib.pyplot as plt

## Energy balance models

This demonstration is based on *energy balance models*.
Specifically, we will be looking at three different papers:
* [North et al. (1981), Energy balance climate models](https://doi.org/10.1029/RG019i001p00091)
* [Flannery (1984), Energy Balance Models Incorporating Transport of Thermal and Latent Energy](https://doi.org/10.1175/1520-0469(1984)041<0414:EBMITO>2.0.CO;2)
* [Roe et al. (2015), The remote impacts of climate feedbacks on regional climate predictability](https://doi.org/10.1038/ngeo2346)

These models aim to describe how the temperature distribution $T$ of the earth varies with latitude, averaging over any variation in the longitudinal direction.
They include at least three effects:
* Latitudinal transport of heat, which we assume is diffusive.
* Absorption of solar radiation, which depends on both insolation and the albedo or reflectivity of the earth.
* Emittance of longwave radiation from earth, which is temperature-dependent through the [Planck law](https://en.wikipedia.org/wiki/Planck%27s_law).

We'll start with the most simplified version of each of these effects and then build up in complexity.
A word of warning: the paper by Flannery prefers to describe spatial dependence of the coefficients in the problem as polynomials in $x$, while the paper by North and others prefers to describe spatial dependence in terms of Legendre polynomials.
You may have to convert back and forth at times.

#### Net balance

The total balance equation states that the flow of heat into and out of a given latitude band has to be balanced by the sum of heat energy coming in and heat energy lost to space.
If $F$ is the heat flux, then the conservation form of our problem is
$$F(x_2) - F(x_1) = \int_{x_1}^{x_2}(a\cdot S - I)dx$$
where $S$ is the solar radiation hitting the earth, $I$ is the emission of longwave radiation.
The coalbedo $a$ dictates how much of the solar radiation is actually absorbed; some will be reflected.
**Exercise**: derive the differential and variational forms of this problem.

#### Diffusive transport

Rather than use the latitude itself as the coordinate, these models use the sine of the latitude, which we'll write as $x$.
This choice of coordinate system means that the expressions for differential operators have to change.
First, the easy part: if $F$ is the heat flux, then
$$\nabla\cdot F = \frac{dF}{dx}.$$
The challenging part is what has to happen to the temperature gradient:
$$\nabla T = (1 - x^2)\frac{dT}{dx}.$$
Again, this extra factor of $1 - x^2$ is because of the transformation to spherical coordinates.
We'll assume at first that the heat flux is proportional to the temperature gradient:
$$F = -D\nabla T$$
for some constant coefficient $D$.
In real applications, we obtain this coefficient by fitting to a combination of global circulation model output and remote sensing data.

Treating heat transport as diffusive is a huge simplification of the general circulation of the ocean and atmosphere.
These simplified models nonetheless have their uses in describing the bulk behavior of the climate.

#### Outgoing longwave radiation

The earth loses heat to space by emission of infrared or longwave radiation.
The hotter the earth surface is, the more energetic the longwave emissions.
The magnitude of the longwave emissions and their temperature sensitivity can be estimated from observational data, giving
$$I(T) = A + BT$$
where A = 203.3 W/m${}^2$ and B = 2.09 W/(m${}^2\; {}^{\circ}$C).
Quoting North et al. 1981, these coefficients are effectively including many complex processes including clouds, infrared-absorbing gases, etc.

#### Incoming shortwave radiation

The earth gains heat from solar radiation, but some amount of this radiation is reflected back to space.
We can approximate the solar input by a parabola (equation 3 in Flannery):
$$S(x) = Q(s_0 - s_2x^2)$$
where Q = 334 W/m${}^2$, s${}_0$ = 1.246, and s${}_2$ = 0.738.

Next, we need to prescribe the co-albedo $a$; the total heat absorbed is equal to $a\cdot S$.
For starters, we can take observed values (equation 18 in North et al, converted to the monomial basis):
$$a(x) = a_0 - a_2x^2$$
where a${}_0$ = 0.782, a${}_2$ = 0.303.

A lot of the interesting physics is wrapped up in the co-albedo.
In particular, the co-albedo is much lower over icy surfaces than over land or ocean -- in other words, $a$ really depends on $T$, which makes the problem nonlinear.
We'll start with the observed values first and then work our way to the nonlinear case.

## All together now

The equation we wish to solve has the conservation form
$$-D(1 - x^2)\frac{dT}{dx}\Bigg|_{x_1}^{x_2} + \int_{x_1}^{x_2}(A + BT - aS)dx = 0$$
for any latitude band $x_1$, $x_2$.
Using the procedure that I showed in class, this problem then has the variational form
$$\int_{-1}^1\left\{D(1 - x^2)\frac{dT}{dx}\frac{d\phi}{dx} + (A + BT - aS)\phi\right\}dx = 0$$
for all test functions $\phi$.
The coalbedo and insolation are
$$a = a_0 - a_2x^2$$
and
$$S = Q(s_0 - s_2x^2).$$
The remaining physical constants and their units are in the table below.

| Name    | Symbol    | Value
| ------- | --------- | --------------
| diffusivity | $D$ | 0.649 W/m${}^2$/${}^\circ$C
| solar constant | $Q$ | 334 W/m${}^2$
| solar flux at eq | $s_0$ | 1.246 
| solar flux curvature | $s_2$ | 0.738
| coalbedo at eq | $a_0$ | 0.782
| coalbedo curvature | $a_2$ | 0.303
| longwave mean | $A$ | 205 W/m${}^2$
| longwave $T$ sensitivity | $B$| 2.23 W/m${}^2$/${}^\circ$C

Here I've made a Python dictionary containing all the physical constants below.
This isn't strictly speaking necessary, but it is good practice to have a *single source of truth* when coding things like this -- all the important input data is in one place rather than scattered everywhere, so you can be sure that none of it gets modified accidentally.

In [ ]:
physical_constants = {
    "diffusivity": 0.649,
    "solar_constant": 334,
    "solar_flux_equator": 1.246,
    "solar_flux_curvature": 0.738,
    "coalbedo_equator": 0.782,
    "coalbedo_curvature": 0.303,
    "mean_longwave": 205,
    "longwave_sensitivity": 2.23,
}

First, we have to create the geometry of the problem we wish to solve.

In [ ]:
num_cells = 64
mesh = firedrake.IntervalMesh(num_cells, -1, +1)

Next we need to choose what basis functions we wish to use to describe the numerical solution.
Here I've chosen a higher-order element family (we'll talk about this more in class).
You might notice in the original papers that they use the basis of Legendre polynomials, which you might remember from when we talked about higher-order collocation methods.

In [ ]:
degree = 4
element = firedrake.FiniteElement("Gauss-Lobatto-Legendre", "interval", degree)
V = firedrake.FunctionSpace(mesh, element)

Now we'll create all the physical parameters: the outgoing longwave radiation, the incoming shortwave radiation, the coalbedo, and the absorbed shortwave.

In [ ]:
x, = firedrake.SpatialCoordinate(mesh)
T = firedrake.Function(V)

D = Constant(physical_constants["diffusivity"])

A = Constant(physical_constants["mean_longwave"])
B = Constant(physical_constants["longwave_sensitivity"])
I = A + B * T

Q = Constant(physical_constants["solar_constant"])
s_0 = Constant(physical_constants["solar_flux_equator"])
s_2 = Constant(physical_constants["solar_flux_curvature"])
S = Q * (s_0 - s_2 * x**2)

a_0 = Constant(physical_constants["coalbedo_equator"])
a_2 = Constant(physical_constants["coalbedo_curvature"])
a = a_0 - a_2 * x**2

And finally we'll create the variational form of the problem.
Observe again how similar it looks to the math.

In [ ]:
ϕ = firedrake.TestFunction(V)
F = (
    (1 - x**2) * D * inner(grad(T), grad(ϕ)) + (I - a * S) * ϕ
) * dx

We don't have to explicitly impose any boundary conditions here; you should think about why that is.

In [ ]:
firedrake.solve(F == 0, T)

The results reproduce the temperatures in the original North paper and are roughly in line with the real values, which suggests that we actually did this correctly.

In [ ]:
fig, ax = plt.subplots()
firedrake.plot(T, axes=ax);

## Ice-albedo feedback

Before, we used the average coalbedo of the earth today, which includes icy regions.
The real coalbedo depends on temperature.
When the annual average temperature is below 0C, you can expect this part of the earth to be ice-covered.
Ice is much more reflective than land or open ocean, so including this effect makes the problem nonlinear:
$$a(x, T) = \begin{cases}a_0 - a_2x^2 & T > T_s \\ b_0 & T < T_s\end{cases}$$
where $T_s$ is a critical temperature -- which might be different from 0C -- and $b_0$ is coalbedo of ice.
See equation 5 in Flannery.

| Name | Symbol | Value
| ---- | ------ | -----
| critical temp | $T_s$ | -10${}^\circ$C
| ice coalbedo | $b_0$ | 0.380

We'll solve this using a basic iterative method.
Before, we had to solve a linear system $L\hat T = f$ for the coefficients $\hat T$.
The nonlinear system we have to solve now, including ice-albedo feedback, has the form
$$L\hat T = f(\hat T).$$
We can approximate it by instead solving
$$L\hat T_{k + 1} = f(\hat T_k),$$
which only involves solving a linear system at each step.

Below, I've rewritten the expression to compute the right-hand side so that it includes the ice-albedo feedback.
To do this, I used the function firedrake.conditional.

In [ ]:
help(firedrake.conditional)

In [ ]:
physical_constants.update(
    {
        "critical_temp": -10.0,
        "ice_coalbedo": 0.380,
    }
)

T_c = Constant(physical_constants["critical_temp"])
a_0 = Constant(physical_constants["coalbedo_equator"])
b_0 = Constant(physical_constants["ice_coalbedo"])

T_n = T.copy(deepcopy=True)
a = firedrake.conditional(T_n <= T_c, b_0, a_0 - a_2 * x**2)

I = A + B * T
F = (
    (1 - x**2) * D * inner(grad(T), grad(ϕ)) + (I - a * S) * ϕ
) * dx

Here I did 10 steps of an iterative method to compute the steady-state temperature.
It's mostly converged after the 3rd stpe.

In [ ]:
Ts = [T.copy(deepcopy=True)]
num_steps = 10
for step in range(num_steps):
    firedrake.solve(F == 0, T)
    T_n.assign(T)
    Ts.append(T.copy(deepcopy=True))

In [ ]:
[firedrake.norm(Ts[n + 1] - Ts[n]) for n in range(num_steps - 1)]

The plot below shows the first three iterations.

In [ ]:
fig, ax = plt.subplots()
coordinates = mesh.coordinates.dat.data_ro[:]
ax.scatter(coordinates, physical_constants["critical_temp"] * np.ones_like(coordinates), 0.5, color="tab:grey")
firedrake.plot(Ts[0], axes=ax, edgecolor="tab:blue", label="0")
firedrake.plot(Ts[1], axes=ax, edgecolor="tab:green", label="1")
firedrake.plot(Ts[2], axes=ax, edgecolor="tab:orange", label="2")
ax.legend();

### Some things to try

See how sensitive the solutions are to changes in the parameters.
Try a time-dependent version and slowly change, say, the longwave feedback.
You could also vary one of the parameters stochastically in time and space and see what the excursions in temperature look like.
With the ice-albedo feedback, can you get a snowball earth?
Gerard has a version that includes moisture and the Clausius-Clapeyron effect.
You could try this on the sphere and change the coalbedo for land surfaces vs ocean.
Aaron Donohoe tells me you'd want to make the diffusivity much higher in the meridional direction.
Aaron also said you could write a paper off of this.
Any of the above would be suitable as a project.
For more inspiration, see [North and Kim](https://www.wiley.com/en-us/Energy+Balance+Climate+Models-p-9783527411320).